# Scoring and Citations Testbed
---

## Import Libraries

In [ ]:
import sys 
import os 
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [2]:
import nltk
import pandas as pd
from nltk.tokenize import sent_tokenize
from nltk.translate.bleu_score import SmoothingFunction, sentence_bleu
from rouge_score import rouge_scorer
from scipy.spatial.distance import cosine
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

/jet/home/iwiryadi/.local/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [3]:
from src.model import load_fo_model, load_ba_model
from src.data import load_cnn_dataset, prepare_cnn_dataset
from src.utils import *
from src.search import *

Using device: cuda


In [4]:
pd.set_option('display.max_colwidth', 200)
pd.set_option('display.float_format', '{:.4f}'.format)

In [5]:
nltk.download("punkt_tab")

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /jet/home/iwiryadi/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [6]:
DEVICE

'cuda'

In [7]:
CACHE_DIR

'/ocean/projects/cis250068p/shared/caches'

## Quick Test

In [8]:
# Load the models
fo_model, fo_tokenizer = load_fo_model()
ba_model, ba_tokenizer = load_ba_model()

config.json:   0%|          | 0.00/598 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/375M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/375M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/146 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.00 [00:00<?, ?B/s]

In [ ]:
fo_tokenizer.pad_token    = "<|padding|>"
fo_tokenizer.pad_token_id = 1   

ba_tokenizer.pad_token    = "<|padding|>"
ba_tokenizer.pad_token_id = 1   

In [9]:
summaries, articles, adverse_summaries = example_texts()

In [12]:
%%time
debug = False

batch = 16

samp_summaries=summaries[:batch]
samp_articles=articles[:batch]
samp_advsummaries=adverse_summaries[:batch]

# Evaluate with batch (normal direction) → S->A (only for Fo model)
base_scores = calculate_score_batch(samp_summaries, samp_articles, fo_model, fo_tokenizer, backward=False, query_direction="normal", debug=debug)

# Evaluate with batch (reverse direction) → A->S
fo_scores = calculate_score_batch(samp_summaries, samp_articles, fo_model, fo_tokenizer, backward=False, query_direction="reverse", debug=debug)
ba_scores = calculate_score_batch(samp_summaries, samp_articles, ba_model, ba_tokenizer, backward=True, query_direction="reverse", debug=debug)

# Adverse summaries
adv_base_scores = calculate_score_batch(samp_advsummaries, samp_articles, fo_model, fo_tokenizer, backward=False, query_direction="normal", debug=debug)
adv_fo_scores = calculate_score_batch(samp_advsummaries, samp_articles, fo_model, fo_tokenizer, backward=False, query_direction="reverse", debug=debug)
adv_ba_scores = calculate_score_batch(samp_advsummaries, samp_articles, ba_model, ba_tokenizer, backward=True, query_direction="reverse", debug=debug)


def extract_metric(scores, key):
    return [s[key] for s in scores]

df = pd.DataFrame({
    'Article': samp_articles,
    'Summary': samp_summaries,
    'Adverse Summary': samp_advsummaries,
    
    'Base_NLL': extract_metric(base_scores, 'normalized_log_prob'),
    'Fo_NLL': extract_metric(fo_scores, 'normalized_log_prob'),
    'Ba_NLL': extract_metric(ba_scores, 'normalized_log_prob'),
    
    'AdvBase_NLL': extract_metric(adv_base_scores, 'normalized_log_prob'),
    'AdvFo_NLL': extract_metric(adv_fo_scores, 'normalized_log_prob'),
    'AdvBa_NLL': extract_metric(adv_ba_scores, 'normalized_log_prob'),
})
df

ValueError: Tokenizer does not have a pad_token_id defined.

In [13]:
%%time

debug = False

summary = summaries[0]
article = articles[0]
adverse_summary = adverse_summaries[0]

# In normal, query is sentence/article, and answer is summary/highlight (S->A direction)
base = calculate_score(summary, article, fo_model, fo_tokenizer, backward=False, query_direction="normal", debug=debug)
# In Fo, query is summary/highlight , and answer is sentence/article(A->S direction)
fo = calculate_score(summary, article, fo_model, fo_tokenizer, backward=False, query_direction="reverse", debug=debug)
# In Ba, query is summary/highlight , and answer is sentence/article(A->S direction)
ba = calculate_score(summary, article, ba_model, ba_tokenizer, backward=True, query_direction="reverse", debug=debug)


adv_base = calculate_score(adverse_summary, article, fo_model, fo_tokenizer, backward=False, query_direction="normal", debug=debug)
adv_fo = calculate_score(adverse_summary, article, fo_model, fo_tokenizer, backward=False, query_direction="reverse", debug=debug)
adv_ba = calculate_score(adverse_summary, article, ba_model, ba_tokenizer, backward=True, query_direction="reverse", debug=debug)


scores_data = {
    'Model Type': ['Base', 'Forward', 'Backward', 'Adv Base', 'Adv Forward', 'Adv Backward'],
    
    'Query Direction': ['S->A', 'A->S', 'A->S', 'S->A', 'A->S', 'A->S'],
    
    'Sequence Log Prob': [
        base['sequence_log_prob'],
        fo['sequence_log_prob'],
        ba['sequence_log_prob'],
        
        adv_base['sequence_log_prob'],
        adv_fo['sequence_log_prob'],
        adv_ba['sequence_log_prob'],
    ],
    'Normalized Log Prob': [
        base['normalized_log_prob'],
        fo['normalized_log_prob'],
        ba['normalized_log_prob'],
        
        adv_base['normalized_log_prob'],
        adv_fo['normalized_log_prob'],
        adv_ba['normalized_log_prob'],
    ],
    'Perplexity': [
        base['perplexity'],
        fo['perplexity'],
        ba['perplexity'],
        
        adv_base['perplexity'],
        adv_fo['perplexity'],
        adv_ba['perplexity'],
    ]
}

CPU times: user 64.9 ms, sys: 321 µs, total: 65.2 ms
Wall time: 64.2 ms


In [14]:
result_df = pd.DataFrame(scores_data)
result_df

,Model Type,Query Direction,Sequence Log Prob,Normalized Log Prob,Perplexity
0,Base,S->A,-48.3527,-3.0220,20.5332
1,Forward,A->S,-113.7687,-2.7748,16.0362
2,Backward,A->S,-117.9552,-2.8770,17.7601
3,Adv Base,S->A,-92.7839,-5.7990,329.9684
4,Adv Forward,A->S,-63.9381,-7.1042,1217.1104
5,Adv Backward,A->S,-57.0505,-6.3389,566.1954


## Linear Search

In [30]:
dataset = load_cnn_dataset(num_samples=1)
dataset = pd.DataFrame(dataset)

Dataset loaded successfully
Example dataset item: {'article': 'LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don\'t think I\'ll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places be

In [31]:
dataset.head(5)

,article,highlights,id
0,"LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell...",Harry Potter star Daniel Radcliffe gets £20M fortune as he turns 18 Monday .\nYoung actor says he has no plans to fritter his cash away .\nRadcliffe's earnings from first five Potter films have be...,42c027e4ff9730fbb3de84c1af0d2c506e41c3e4


In [32]:
all_pairs, meta = prepare_cnn_dataset(dataset)

In [34]:
linear_results = linear_attribution_search_batched(all_pairs, meta, fo_model, fo_tokenizer, ba_model, ba_tokenizer)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [35]:
linear_results

[{'id': '42c027e4ff9730fbb3de84c1af0d2c506e41c3e4',
  'highlight': 'Harry Potter star Daniel Radcliffe gets £20M fortune as he turns 18 Monday .',
  'base_citation': 'All rights reserved.This material may not be published, broadcast, rewritten, or redistributed.',
  'base_score': -2.009256676398231,
  'base_perplexity': 7.45777174872994,
  'fo_citation': "LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him.",
  'fo_score': -3.607265502139714,
  'fo_perplexity': 36.86510730304163,
  'ba_citation': "LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him.",
  'ba_score': -2.715530435206676,
  'ba_perplexity': 15.112624214164343}]

In [ ]:
linear_results = linear_attribution_search(dataset, fo_model, fo_tokenizer, ba_model, ba_tokenizer)

  0%|          | 0/100 [00:00<?, ?it/s]

In [26]:
binary_results = binary_search_attribution_search(dataset, fo_model, fo_tokenizer, ba_model, ba_tokenizer)

  0%|          | 0/100 [00:00<?, ?it/s]

In [27]:
exclusion_results = exclusion_search_attribution_search(dataset, fo_model, fo_tokenizer, ba_model, ba_tokenizer)

  0%|          | 0/100 [00:00<?, ?it/s]

In [26]:
def calculate_embedding_similarity(highlight, citation):
    """Calculate cosine similarity between sentence embeddings."""
    model = SentenceTransformer('all-MiniLM-L6-v2')
    
    # Generate embeddings
    highlight_embedding = model.encode([highlight])[0]
    citation_embedding = model.encode([citation])[0]
    
    # Calculate cosine similarity (1 - cosine distance)
    similarity = 1 - cosine(highlight_embedding, citation_embedding)
    return similarity

def calculate_rouge_score(highlight, citation):
    """Calculate ROUGE-L F-measure score."""
    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    scores = scorer.score(highlight, citation)
    return scores['rougeL'].fmeasure

def calculate_tfidf_score(highlight, citation):
    """Calculate TF-IDF similarity score."""
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([highlight, citation])
    
    # Convert sparse matrix to dense array for cosine similarity calculation
    dense_matrix = tfidf_matrix.toarray()
    
    # Calculate cosine similarity
    similarity = 1 - cosine(dense_matrix[0], dense_matrix[1])
    return similarity

def process_data(data):
    """Process the data and reformat to show scores by citation type in columns."""
    results = []
    
    for item in data:
        highlight = item['highlight']
        result_entry = {'id': item['id'], 'highlight': highlight}
        
        # Process each citation type
        for citation_type in ['base_citation', 'ba_citation', 'fo_citation']:
            prefix = citation_type.split('_')[0]  # Extract 'base', 'ba', or 'fo'
            
            if citation_type in item and item[citation_type]:
                citation = item[citation_type]
                
                # Calculate scores
                emb_similarity = calculate_embedding_similarity(highlight, citation)
                rouge_score = calculate_rouge_score(highlight, citation)
                tfidf_score = calculate_tfidf_score(highlight, citation)
                
                # Add scores as columns with prefix
                result_entry[f'{prefix}_emb_similarity'] = emb_similarity
                result_entry[f'{prefix}_rouge_score'] = rouge_score
                result_entry[f'{prefix}_tfidf_score'] = tfidf_score
                
            else:
                # Set default values if citation doesn't exist
                result_entry[f'{prefix}_emb_similarity'] = None
                result_entry[f'{prefix}_rouge_score'] = None
                result_entry[f'{prefix}_tfidf_score'] = None
                
        results.append(result_entry)
    
    return results

# Process the data
linear_final_results = process_data(linear_results)
binary_final_results = process_data(binary_results)
exclusion_final_results = process_data(exclusion_results)


NameError: name 'binary_results' is not defined

In [29]:
r = pd.DataFrame(linear_final_results) 
r.drop(['id', 'highlight'], axis=1).mean()

base_emb_similarity   0.4150
base_rouge_score      0.1879
base_tfidf_score      0.1701
ba_emb_similarity     0.6442
ba_rouge_score        0.3303
ba_tfidf_score        0.2898
fo_emb_similarity     0.6456
fo_rouge_score        0.3480
fo_tfidf_score        0.3144
dtype: float64

In [30]:
r = pd.DataFrame(binary_final_results) 
r.drop(['id', 'highlight'], axis=1).mean()

base_emb_similarity   0.4260
base_rouge_score      0.1673
base_tfidf_score      0.1448
ba_emb_similarity     0.6403
ba_rouge_score        0.3137
ba_tfidf_score        0.2745
fo_emb_similarity     0.6378
fo_rouge_score        0.3302
fo_tfidf_score        0.2923
dtype: float64

In [31]:
r = pd.DataFrame(exclusion_final_results) 
r.drop(['id', 'highlight'], axis=1).mean()

base_emb_similarity   0.4007
base_rouge_score      0.1688
base_tfidf_score      0.1433
ba_emb_similarity     0.6219
ba_rouge_score        0.3161
ba_tfidf_score        0.2895
fo_emb_similarity     0.6041
fo_rouge_score        0.3204
fo_tfidf_score        0.2921
dtype: float64

In [32]:
df_linear = pd.DataFrame(linear_final_results)
df_binary = pd.DataFrame(binary_final_results)
df_exclusion = pd.DataFrame(exclusion_final_results)

df_merged = pd.merge(df_linear, df_binary, on=['id', 'highlight'],suffixes=('_linear', '_binary'))

df_exclusion = df_exclusion.add_suffix('_exclusion')
df_exclusion = df_exclusion.rename(columns={'id_exclusion': 'id', 'highlight_exclusion': 'highlight'})

df_merged = pd.merge(df_merged, df_exclusion, on=['id', 'highlight'])

mean_series = df_merged.drop(['id', 'highlight'], axis=1).mean()

data = {
    'Base_linear': [
        mean_series['base_emb_similarity_linear'],
        mean_series['base_rouge_score_linear'],
        mean_series['base_tfidf_score_linear']
    ],
    'Fo_linear': [
        mean_series['fo_emb_similarity_linear'],
        mean_series['fo_rouge_score_linear'],
        mean_series['fo_tfidf_score_linear']
    ],
    'Ba_linear': [
        mean_series['ba_emb_similarity_linear'],
        mean_series['ba_rouge_score_linear'],
        mean_series['ba_tfidf_score_linear']
    ],
    'Base_binary': [
        mean_series['base_emb_similarity_binary'],
        mean_series['base_rouge_score_binary'],
        mean_series['base_tfidf_score_binary']
    ],
    'Fo_binary': [
        mean_series['fo_emb_similarity_binary'],
        mean_series['fo_rouge_score_binary'],
        mean_series['fo_tfidf_score_binary']
    ],
    'Ba_binary': [
        mean_series['ba_emb_similarity_binary'],
        mean_series['ba_rouge_score_binary'],
        mean_series['ba_tfidf_score_binary']
    ]
}

table_df = pd.DataFrame(data, index=['Embedding', 'Rouge', 'Tfidf'])
table_df

,Base_linear,Fo_linear,Ba_linear,Base_binary,Fo_binary,Ba_binary
Embedding,0.4425,0.6380,0.6453,0.4260,0.6378,0.6403
Rouge,0.1945,0.3206,0.3099,0.1673,0.3302,0.3137
Tfidf,0.1664,0.2855,0.2708,0.1448,0.2923,0.2745
